In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

### Import 

In [3]:
ROOT = Path().cwd().parent
ROOT

PosixPath('/Users/thomassimader/Documents/NIRB')

In [4]:
import sys
sys.path.append(str(ROOT))
from ComsolClasses.comsol_classes import COMSOL_VTU

In [5]:
import_folder = ROOT / "Snapshots" / "01" / "Exports" 
import_folder.exists()

True

In [6]:
temperatures = np.load(import_folder / "temperatures.npy" )
temperatures_diff = np.load(import_folder / "temperatures_diff.npy" )

In [7]:
comsol_data  = COMSOL_VTU("/Users/thomassimader/Documents/NIRB/Snapshots/01/Training/Training_000.vtu")

In [8]:
N_SNAPS, N_TIMES, N_POINTS = temperatures_diff.shape

colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])

fig = go.Figure()
for idx_snap, temperature_array in enumerate(temperatures_diff):
# customdata=np.stack(([t_h] * len(comsol_data.times), [host_k] * len(comsol_data.times)) , axis=-1),
    input_args = {'x' : list(comsol_data.times.values()),
                #   'customdata' : customdata,
                #   'hovertemplate' : "Time: %{x:.2e}<br>Temperature: %{y:.2f}<br>T_h: %{customdata[0]:.2f}<br><br>host_k: %{customdata[1]:.2e}<br>",
                    'legendgroup' : f"{idx_snap:03d}",
                    'name' : f"{idx_snap:03d}",
                    'line' : dict(color=colors[idx_snap]),
                    'opacity': 0.4}
    fig.add_trace(go.Scatter(y=np.max(temperature_array, axis=1), **input_args))
    fig.add_trace(go.Scatter(y=np.min(temperature_array, axis=1), **input_args))

fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title="Temperature Difference (T-T0)",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Temperature [K]"     # Optional: label for y-axis
)

fig.show()


In [13]:
sim_times_path = import_folder / "sim_times.npy"
sim_times = np.load(sim_times_path)

In [25]:
fig = go.Figure(data=[go.Histogram(
    x=sim_times / 60,
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        # start=-4.0,
        # end=3.0,
        size=2,
        ),
    marker_color='#EB89B5',
    opacity=0.75,
     texttemplate="%{x}"
    )])

fig.update_layout(
    title_text=f'{sim_times_path.stem}', # title of plot
    xaxis_title_text='Time [min]', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargroupgap=0.1, # gap between bars of the same location coordinates
)

fig.show()
